In [ ]:
import os
import matplotlib.pyplot as plt
GPU_NO = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NO
TRANSFORMERS_CACHE='/workspace/HF_cache/transformers_cache/'
import torch

In [ ]:
MGIT_PATH=os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
import sys
sys.path.append(MGIT_PATH)
from utils.lineage.graph import *
from utils import meta_functions

In [ ]:
preprocess_file = MGIT_PATH + '/utils/preprocess_utils.py'
user_file=os.getcwd()+'/user_functions.py'
perturbation_file = MGIT_PATH + '/utils/perturbation_utils.py'

In [ ]:
!rm -rf tmp_MTL_cola
!rm -rf tmp_MTL_sst2
!rm -rf tmp_MTL_mlm
!rm -rf parameter_store

In [ ]:
"""
g = LineageGraph()
g.save('./')
"""
!../../../mgit init

In [ ]:
"""
g.load('./')
root = LineageNode.load_from_file('./root.json')

g.add(root)
g.save('./')
"""
!../../../mgit add -node_config ./root.json

In [ ]:
"""
g.load('./')
cola_node = LineageNode.load_from_file('./cola_node.json')

g.add(cola_node,parent='tmp_MTL_mlm',etype='adapted')

g.save('./')
"""
!../../../mgit add -node_config ./cola_node.json -parent tmp_MTL_mlm -e adapted

In [ ]:
"""
g.load('./')
sst2_node = LineageNode.load_from_file('./sst2_node.json')

g.add(sst2_node,etype='adapted',parent='tmp_MTL_mlm')

n = g.show()

g.save('./')
"""
!../../../mgit add -node_config ./sst2_node.json -parent tmp_MTL_mlm -e adapted

In [ ]:
g = LineageGraph.load_from_file('./')
assert set(['tmp_MTL_sst2', 'tmp_MTL_cola']) == set([entry.output_dir for entry in g.entanglement_tracker.get_entangled(g.nodes['tmp_MTL_sst2'])])

In [ ]:
g = LineageGraph.load_from_file('./')
assert len(g.entanglement_tracker.temporary_nodes) == len(g.nodes)

In [ ]:
!../../../mgit downward_train -n tmp_MTL_sst2

In [ ]:
g = LineageGraph.load_from_file('./')
assert torch.equal(g.nodes['tmp_MTL_sst2'].get_model().model.roberta.encoder.layer[0].attention.self.query.weight, g.nodes['tmp_MTL_cola'].get_model().model.roberta.encoder.layer[0].attention.self.query.weight)

In [ ]:
"""
g = LineageGraph.load_from_file('./')
g.nodes['tmp_MTL_sst2'].add_test_from_file('./sst2_test.json')
g.save('./')
"""
!../../../mgit test -n tmp_MTL_sst2 -test_config ./sst2_test.json -exe

In [ ]:
"""
g = LineageGraph.load_from_file('./')
g.nodes['tmp_MTL_cola'].add_test_from_file('./cola_test.json')
g.save('./')
"""
!../../../mgit test -n tmp_MTL_cola -test_config ./cola_test.json -exe

In [ ]:
g = LineageGraph.load_from_file('./')
assert g.nodes['tmp_MTL_sst2'].is_test_failure() is False

In [ ]:
g = LineageGraph.load_from_file('./')
assert g.nodes['tmp_MTL_cola'].is_test_failure() is False

In [ ]:
g = LineageGraph.load_from_file('./')
assert set(['tmp_MTL_sst2', 'tmp_MTL_cola']) == set([entry.output_dir for entry in g.entanglement_tracker.get_entangled(g.nodes['tmp_MTL_cola'])])

In [ ]:
!../../../mgit display_test -metrics

In [ ]:
res = !du -s tmp_MTL_cola
cola_store = int(res[0].split('\t')[0])

res = !du -s tmp_MTL_sst2
sst2_store = int(res[0].split('\t')[0])

res = !du -s tmp_MTL_mlm
mlm_store = int(res[0].split('\t')[0])

res = !du -s parameter_store
global_store = int(res[0].split('\t')[0])

print('storage savings:',(cola_store+sst2_store+mlm_store)/global_store)

In [ ]:
!rm -rf tmp_MTL_cola
!rm -rf tmp_MTL_sst2
!rm -rf tmp_MTL_mlm
!rm -rf parameter_store